### Input Instructions
* You need to first train and save the model:-
  1. Download book.csv.
  2. Change file_path variable to path of book.csv.
  3. Also change the model_path accordingly to save the model
* Now change the model_saved_path to the directory where the model is saved.
* other than this specify a resume_path which has the path to the resume to highlight
* And output_pdf_path as the output path of the pdf, also specify the name of the pdf to be made.
* Example_output pdf is the pdf we ran on and outputted our result.

Note :- You can use find these variables using Ctrl +F in Vs Code or in most code editors.
 

In [37]:
pip install pdfplumber


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 35.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 45.7 MB/s eta 0:00:00:00:01
Note: you may need to restart the kernel to use updated packages.


In [41]:
pip install PyMuPdf


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 27.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 33.4 MB/s eta 0:00:00:00:010:01
Note: you may need to restart the kernel to use updated packages.


In [35]:
import pandas as pd
import numpy as np
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split

# Load the DataFrame
file_path = '/kaggle/input/resumes/book.csv'
df = pd.read_csv(file_path)  # Adjust separator based on your CSV

# Define queries and keywords for hypothetical label assignment
queries = ['Frontend', 'Backend', 'Mathematics', 'Programming', 'Leadership', 'Accounting', 'AI', 'ML']
category_keywords = {
    'Frontend': ['html', 'css', 'javascript', 'react', 'vue', 'angular'],
    'Backend': ['node.js', 'django', 'flask', 'ruby on rails', 'express'],
    'Mathematics': ['algebra', 'calculus', 'statistics', 'geometry', 'probability'],
    'Programming': ['python', 'java', 'c++', 'go', 'rust', 'kotlin'],
    'Leadership': ['management', 'team lead', 'director', 'head', 'supervisor'],
    'Accounting': ['taxes', 'accounts receivable', 'ledger', 'financial statements', 'bookkeeping'],
    'AI': ['artificial intelligence', 'neural networks', 'deep learning', 'nlp', 'computer vision'],
    'ML': ['machine learning', 'svm', 'random forest', 'k-means', 'regression']
}

def assign_query_and_label(text):
    query = np.random.choice(queries)
    keywords = category_keywords[query]
    label = 1 if any(keyword in text.lower() for keyword in keywords) else 0
    return query, label

# Apply function to DataFrame
df['Query'], df['Label'] = zip(*df['Resume_str'].apply(assign_query_and_label))

# Split into train and eval DataFrames
train_df, eval_df = train_test_split(df, test_size=0.2, random_state=42)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def preprocess_function(examples):
    # Tokenize the inputs (text) and include the labels
    tokenized_inputs = tokenizer(examples['Resume_str'], truncation=True, padding='max_length', max_length=512)
    tokenized_inputs["labels"] = examples["Label"]  # Ensure 'Label' column exists and is correctly named
    return tokenized_inputs

# Convert DataFrames to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)

# Apply the tokenization preprocessing, ensuring labels are included
train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

# Tokenizer and Model
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Training arguments
training_args = TrainingArguments(
    output_dir='./models/',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs/',
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Start training
trainer.train()

model_path = '/kaggle/working/models'
model.save_pretrained(model_path)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss


In [46]:
import torch
import pdfplumber
import re
import fitz
from transformers import BertTokenizer, BertModel

# Load your previously trained tokenizer and model
model_saved_path = BertModel.from_pretrained('/kaggle/working/models')

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()
    return text

def find_keywords_in_resume(resume_text, query):
    # Tokenize input text and query
    inputs = tokenizer(query, resume_text, return_tensors='pt', padding=True, truncation=True)

    # Get BERT embeddings
    with torch.no_grad():
        outputs = model(**inputs)
        query_embedding = outputs.last_hidden_state[:, 0, :]  # Query embedding
        resume_embeddings = outputs.last_hidden_state[:, 1:, :]  # Resume embeddings

    # Compute cosine similarity between query and each token in resume text
    similarities = torch.nn.functional.cosine_similarity(query_embedding, resume_embeddings, dim=-1)

    # Sort tokens by similarity and extract keywords
    sorted_indices = torch.argsort(similarities, descending=True, dim=-1)
    keywords = [tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][i].item()) for i in sorted_indices[0][:10]]  # Extract top 10 keywords

    # Filter out keywords containing non-alphanumeric characters
    keywords = [word for word in keywords if re.match("^[a-zA-Z]+$", word)]
    return keywords

def highlight_words_in_pdf(pdf_path, words_to_highlight, output_pdf_path):
    # Open the PDF
    doc = fitz.open(pdf_path)
    
    for page in doc:
        # Search for each word on the page and highlight it
        for word in words_to_highlight:
            text_instances = page.search_for(word)
            
            # Highlight each instance found
            for inst in text_instances:
                highlight = page.add_highlight_annot(inst)
                highlight.update()
    
    # Save the highlighted PDF
    doc.save(output_pdf_path)

# Example usage
resume_path = "/kaggle/input/resume-dataset/data/data/ACCOUNTANT/10554236.pdf"
query = "leadership"
resume_text = extract_text_from_pdf(resume_path)
keywords = find_keywords_in_resume(resume_text, query)
print("Keywords related to query '{}' in the resume: {}".format(query, keywords))

output_pdf_path = "/kaggle/working/new_file.pdf"
highlight_words_in_pdf(resume_path, keywords, output_pdf_path)


Keywords related to query 'leadership' in the resume: ['defense', 'software', 'leadership']
